# Unified LLM Generation Pipeline

This notebook provides an interface to the unified generation pipeline.
All logic lives in `generation.py` - this notebook is for interactive testing and batch runs.

The pipeline handles both Paleolithic and Holocene people automatically based on the sampled birth year.

In [1]:
import dill
from tqdm import tqdm

from generation import (
    generate_person,
    generate_batch,
    generate_batch_parallel,
    # Individual steps if needed
    generate_geography,
    generate_demographics,
    generate_structured_incidents,
    generate_historical_context,
    generate_name,
    generate_narrative_plan,
    generate_narrative,
    run_pipeline,
    reset_to_stage
)
from llm_utils import GenerationContext, extract_json

from person import sample_year, sample_person, Person

import copy

# Batch Generation

In [6]:
test_people = generate_batch_parallel(n=70, model="gpt-5.2", workers=30)

Sampling 70 people...


Warning 3: Cannot find header.dxf (GDAL_DATA is not defined)


Generating 70 people with 30 parallel workers...


  0%|                                                    | 0/70 [00:00<?, ?it/s]

100%|███████████████████████████████████████████| 70/70 [12:02<00:00, 10.33s/it]

Done: 70 generated


In [7]:
new_people = people + test_people

In [9]:
'''with open('new_people.pkl', 'wb') as f:
    dill.dump(new_people,f)
%run export.py new_people.pkl'''

Loading people from new_people.pkl...
Found 100 people
Removing existing markdown files from ../_lives...
  Removed 30 files
  Exported 10 people...
  Exported 20 people...
  Exported 30 people...
  Exported 40 people...
  Exported 50 people...
  Exported 60 people...
  Exported 70 people...
  Exported 80 people...
  Exported 90 people...
  Exported 100 people...

Successfully exported 100 people to ../_lives


In [10]:
with open('new_people.pkl', 'rb') as f:
    people = dill.load(f)

In [11]:
print(people[10].narrative)

Hanna was born on November 26, 1242, in the hills above Sidon, in a Melkite village that prayed in the Byzantine rite and spoke Arabic in the lanes and fields. The coast belonged to a Crusader lordship, and the countryside lived with tolls, requisitions, and rumors of armies moving between the strongholds and the interior.

His father, Butrus, worked stone and wood when there was building to do—repairs to terraces, a new lintel for a doorway, a patched wall for a neighbor—then turned back to the family’s small plot. His mother, Mariam, ran the house: grinding, baking, spinning, tending a few animals, and keeping a corner with an icon and a little lamp that she lit on feast days. Hanna grew up hearing Greek words mixed into prayer and church talk; by adolescence he could hold a conversation with Greek-speaking clerics and travelers on the coastal road, even though he read only slowly and could not write more than a clumsy mark.

His oldest sister Maryam, born in 1239, mothered the young